# Tarea 1


### Integrantes

*   Arath Alejandro Reyes López
*   Jesús Eduardo García Tapia (El Yosus)
*   Valeria Lucero Cruz Estrada
*   José Antonio Ramirez Maciel
*   Juan Manuel Hernández Acosta




## Pendientes

*   Falta la opción de con alisado (ya esta )
*   Falta la valoración de bonos
*   Falta el VaR y CVaR de bonos



In [ ]:
import pandas as pd
import numpy as np
!pip install yfinance
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
# import selenium # Por si hay que bajar las series de la BMV

In [ ]:
def CVaR(x , VaR):
  return np.mean(x[x >= VaR])

1-  Calcular el VaR y CVaR a un día por los métodos Simulación histórica con y sin alisado. Tomar como día de valoración el 31 de marzo de 2022.

# Acciones

1.1 1000 acciones de GCarso, -5000 de America Móvil y 1200 WalMart. Todas de la BMV.

In [ ]:
class Riesgo_Equity_FX:

  def __init__(self):
    self.posiciones = None
    self.acciones = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.ordenado = None # VaR ordenado (ver código de Dani y Fer, Yosus grabó la clase)
    self.alisado = None # Implementación con alisado (pendiente)
  
  def compute(self,acciones, posiciones, fecha_valoracion, alpha = 0.98 ,ordenado = False, alisado = False):
    # Deberían de ser de BMV (pendiente)
    stock_data = yf.download(acciones, end = fecha_valoracion)

    X=stock_data.Close
    X0=stock_data.Close.tail(1)
    DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))
    Y0=X0.squeeze()*posiciones
    X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
    Y_s=X_s.squeeze()*posiciones
    PL=Y_s-Y0 # Seguro?
    cols=PL.shape[1]
    PL['Total']=PL.sum(axis=1)
    VaR=PL.quantile(alpha, numeric_only=False)
    VaR_C=[]

    for i in range(0,PL.shape[1]):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = posiciones
    self.acciones = acciones
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha

    return


In [ ]:
misAcciones = Riesgo_Equity_FX()

In [ ]:
misAcciones.compute(['GCARSOA1.MX','AMXL.MX','WALMEX.MX'], [1000, -5000, 1200], '2022-03-31', 0.95)

[*********************100%***********************]  3 of 3 completed


In [ ]:
misAcciones.summary

,VaR,CVaR
AMXL.MX,466.903991,1589.019551
GCARSOA1.MX,11141.322091,17872.772896
WALMEX.MX,2604.423898,3691.978530
Total,10606.275624,17561.975086


# Divisas

1.2 Divisas

In [ ]:
misDivisas = Riesgo_Equity_FX()

In [ ]:
# Verificar lo del cambio a peso (ej. EURUSD=X a Pesos Mexicanos) # 
misDivisas.compute(['USDMXN=X', 'EURMXN=X', 'GBPMXN=X'], [1500, 700, -600], '2022-03-31')

[*********************100%***********************]  3 of 3 completed


In [ ]:
misDivisas.summary

,VaR,CVaR
EURMXN=X,723.134680,1051.527721
GBPMXN=X,308.988191,466.965161
USDMXN=X,185.412190,258.621760
Total,856.116966,1219.829955


# Fixed Income

1.3 2 bonos largos, uno cupón cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 días, un bono M con tasa fija de 6.5% con un cupon de 6.05% anual vencimiento de 3600 días y nocional de 1000, y un bono corto 1000 bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente).

In [ ]:
def Cete(tasa, t): # días del bono t = (fecha_entrega - fecha_valoración).days
  return 10/(1 + tasa*(t/360))

In [ ]:
def BonoM(x, plazos, plazocupon, tfcupon, nominal, contratos):
  x=x0  
  N=int(plazos/plazocupon)+1   
  a=(1-(1+x*plazocupon/360)**(-N))/(plazocupon*x/360)
  p1=plazos-plazocupon*(N-1)   
  precio=((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)**N))*(1+x*plazocupon/360)**(1-p1/plazocupon) 

  return precio

In [ ]:
def BondeD():
  pass

In [ ]:
class Riesgo_Fixed_Income:

  def __init__(self):
    self.posiciones = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    
  def compute(self, cete, bonom, bonde, alpha):
    """
    Donde cete, bonom, bonde son diccionarios con las siguientes características:
    cete = {'Posicion':posicion_c, 'Plazo':plazo, 'Tasa':curva_tasa}
    bonom = {'Posicion:'posicion_bm, 'Tasa_fija':fija,'Tasa_flotante':flotante, 'Plazo_fijo':plazo_fijo,\
    'Plazo_acum':plazo_acum}
    bonde = {'Posicion':posicion_bd, 'Tasas':tasas_df}
    """
    posiciones = [cete['Posicion'],bonom['Posicion'],bonde['Posicion']]
    # Pegar las tasas con las fechas adecuadas
    X = cete['Tasa'][['Fecha',str(cete['Plazo'])]].merge(bonom['Tasa'],how = 'inner',on= 'Fecha') # Combinar tasas
    X = X.merge(bonde['Tasas'],how = 'inner',on = 'Fecha')

    X0 = X.head(1)
    DeltaX = np.log(X.shift(-1, fill_value=1)/X)
    X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
    # Y_s = ['Cete', 'BonoM','BondesD']
    # Y_s = aplicar valoracion de cete a lo de cete, bonom a lo de bonom y bonde a lo de bonde
    Y0 = Y_s.head(1)
    PL = Y0 - Y_s
    cols=PL.shape[1]
    PL['Total']=PL.sum(axis=1)
    VaR=PL.quantile(alpha, numeric_only=False)
    VaR_C=[]

    for i in range(0,PL.shape[1]):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = {'Cete':cete['Posicion'],'BonoM':bonom['Posicion'],'BondeD':bonde['Posicion']}
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    return



################################################################
  def compute(self, curva_tasa, fval, fent, posicion, alpha):
    """
    - curva_tasa: DataFrame
    """
    # CETE
    self.plazo = (fent - fval).days # REVISAR
    ult_tasa = curva_tasa[str(self.plazo)][0]
    df = pd.DataFrame({'Tasa':curva_tasa[str(self.plazo)]})
    df['Ret_Tasa'] = (df['Tasa'].shift(-1, fill_value=0)/df['Tasa']) - 1
    df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])
    df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = (self.plazo))
    ult_cete = df['CETE'][0]
    df['CETE_PL'] = ult_cete - df['CETE']
    self.VaR = df['CETE_PL'].quantile(alpha, numeric_only=False)
    self.CVaR = CVaR(df['CETE_PL'],self.VaR)
    self.alpha = alpha
    self.posicion = posicion
    self.fecha_valoracion = fval
    self.fecha_entrega = fent
    # Bonos M

    # Bo

    return

In [ ]:
class Riesgo_Cete:

  def __init__(self):
    self.posicion = None
    self.fecha_valoracion = None
    self.fecha_entrega = None
    self.plazo = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    # self.summary = None
    
  
  def compute(self, curva_tasa, fval, fent, posicion, alpha):
    """
    - curva_tasa: DataFrame
    """
    # CETE
    self.plazo = (fent - fval).days # REVISAR
    ult_tasa = curva_tasa[str(self.plazo)][0]
    df = pd.DataFrame({'Tasa':curva_tasa[str(self.plazo)]})
    df['Ret_Tasa'] = (df['Tasa'].shift(-1, fill_value=0)/df['Tasa']) - 1
    df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])
    df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = (self.plazo))
    ult_cete = df['CETE'][0]
    df['CETE_PL'] = ult_cete - df['CETE']
    self.VaR = df['CETE_PL'].quantile(alpha, numeric_only=False)
    self.CVaR = CVaR(df['CETE_PL'],self.VaR)
    self.alpha = alpha
    self.posicion = posicion
    self.fecha_valoracion = fval
    self.fecha_entrega = fent
    # Bonos M

    # Bo

    return

# Futuros

1.4 
100 contratos  de compra con futuros de peso dolar con un strike de 20.83 vencimiento de 5 dias y 50 contratos de venta de futuros del IPC con strike de 49525 vencimiento de 53 dias.


In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 768, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 768 (delta 123), reused 0 (delta 0), pack-reused 579
Receiving objects: 100% (768/768), 41.17 MiB | 28.20 MiB/s, done.
Resolving deltas: 100% (502/502), done.
Checking out files: 100% (409/409), done.


In [ ]:
data_PD = yf.download('USDMXN=X', end = '2022-03-31')
data_IPC = yf.download('^MXX', end = '2022-03-31')
data_libor ="RiesgosFinancieros/2020-2/Insumos/tasa_libor.txt"
X=data_PD.Close
X0=data_PD.Close.tail(1)
Y=data_IPC.Close
Y0=data_IPC.Close.tail(1)

In [ ]:
def forwardTDC(self, t, tl, tn, ,ks , M): #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot, M= monto, k= strike
  f = s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  F= M*((f-k)/(1+t*tn/360))
  return


In [ ]:
def forardInd(self, t, td, tn, s ,k, M,N):
  f= s*(1+((tn-td)*t/360))
  F= M*N*((f-k)/(1+t*tn/360))
  return

# Interest Rate Swap

1.5  Un swap largo con nocional de 16 millones pagando tasa fija de 6.6% anual (cada 28 días) y recibiendo tasa flotante de la curva TIIE vencimiento de 588 días, un swap corto con nocional de 12 millones pagando tasa variable de la TIIE y recibiendo fijja de 5.9% vencimiento de 270 días.

In [ ]:
def InterpolacionLineal(x,X,Y):
    n = len(X)
    for i in range(n-1):
        if X[i]<= x <= X[i+1]:
            a = Y[i+1] - Y[i]
            b = (X[i+1] - X[i]).days
            return Y[i] + (x - X[i]).days*(a/b)
    return "No es posible interpolar"

class InterestRateSwap:
    
    def __init__(self):
        self.precio = None
        self.n_cupones = None
        self.nocional = None
        self.posicion = None
        self.convencion = None
        self.fija = None
        self.flotante = None
        self.descuentos = None
        self.summary = None
        
    def compute(self, fval, M, n, fix_rate, flotante,calen_flotante, descuentos, calen_desc, 
                plazos, tau,  payer = False, act_360 = True):
        """
        M: Nocoinal
        n: # de cupones
        fix_rate: Tasa fija de la pata larga del IRS
        flotante: Curva de tasas de mercado (la que encuentras en BANXICO) de la pata corta
        calen_flotante: Las fechas dadas por la curva de flotante
        descuentos: Curca de tasas de descuento de mercado para valuar el IRS
        calen_desc: Las fechas dadas por la curva de descuentos
        plazos : Lista de plazo del i -ésimo cupón (habiles)
        tau: Plazo en dias del i-ésimo cupón. (naturales)
        payer: Dummy; 1 si paga fija, 0 si paga flotante
        act_360: Dummy; tipo de convención 1 si act/360, 0 si act/365
        """

        spot = fval
        
        if act_360:
            conv = 360
        else:
            conv = 365
            
        aux = pd.DataFrame({"Cupon":list(range(1,n+1)), "Tau":tau})
        aux["Desde Spot"] = aux["Tau"].cumsum()
        aux["Fecha"] = aux["Desde Spot"].apply(lambda x: timedelta(x) + spot)
                
        df = pd.DataFrame({"Cupon":list(range(1,n+1)), "Plazo": plazos, "Tau": tau})
                
        df["Fechas"] =  aux["Fecha"]
        df["Flotante"] = df["Fechas"].apply(InterpolacionLineal, args=(calen_flotante, flotante))
        df["Descuentos"]=df["Fechas"].apply(InterpolacionLineal, args=(calen_desc, descuentos))
        
        # Calculo de tasas forward
        df['Forward_Flotante'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante']*aux['Desde Spot']/conv))/(1+(df['Flotante'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante'][0] = df['Flotante'][0]
        
        df["Sumando"] = ((df["Forward_Flotante"]-fix_rate)*df["Plazo"]/conv) / (1 + (df["Descuentos"]*df["Tau"]/conv))
        
        self.precio =  M*((-1)**payer)*df["Sumando"].sum()
        self.n_cupones = n
        self.nocional = M
        self.posicion = "Payer" if payer else "Receiver"
        self.convencion = conv
        self.fija = fix_rate
        self.flotante = df[["Fechas","Forward_Flotante"]]
        self.descuentos = df[["Fechas", "Descuentos"]]
        self.summary = df
        
        return
    
    def plots(self):
        Bool = True
        while Bool:
            print("\n")
            choice = input("Si deseas ver las tasas de descuento teclea 'Descuentos', si deseas\
                                ver las tasas flotantes teclea 'Tasas' o si deseas salir escribe '0':\n")
            if choice == 'Descuentos' or choice =='Tasas' or choice == '0':
                Bool = False
            else:
                print('\nTu elección no es válida')
        if choice == 'Descuentos':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Descuentos'])
            ax.set_title("Curva de Descuentos",fontsize = '25')
            plt.show()
            
            return
        elif choice == 'Tasas':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Forward_Flotante'], color = 'red')
            ax.set_title("Curva de Tasas Forward Flotantes",fontsize = '25')
            plt.show()
            
            return
        else:
            return

In [ ]:
class Riesgo_IRS:

  def __init__(self):
    self.posiciones = None
    self.tasas_fijas = None
    self.vencimientos = None
    self.dias_bono = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    
  def compute(self, fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c, vencimientos, dias_bono, posiciones, alpha):
    """
    a
    """
    calen_flotante_l = cupon_l.columns
    calen_flotante_l = list(calen_flotante_l)
    calen_flotante_l.remove('DATE')
    calen_desc_l = descuento_l.columns
    calen_desc_l = list(calen_desc_l)
    calen_desc_l.remove('DATE')


    calen_flotante_c = cupon_c.columns
    calen_flotante_c = list(calen_flotante_c)
    calen_flotante_c.remove('DATE')
    calen_desc_c = descuento_c.columns
    calen_desc_c = list(calen_desc_c)
    calen_desc_c.remove('DATE')

    # ya
    n_l = int(vencimientos[0]/dias_bono[0])
    if n_l == (vencimientos[0]/dias_bono[0]):
        plazos_l = [dias_bono[0]]*n_l
        tau_l = plazos_l
    else:
        plazos_l = [vencimientos[0]-n_l*dias_bono[0]] + [dias_bono[0]]*(n_l-1)
        tau_l = plazos_l

    n_c = int(vencimientos[1]/dias_bono[1])
    if n_c == (vencimientos[1]/dias_bono[1]):
        plazos_c = [dias_bono[1]]*n_c
        tau_c = plazos_c
    else:
        plazos_c = [vencimientos[1]-n_c*dias_bono[1]] + [dias_bono[1]]*(n_c-1)
        tau_c = plazos_c



    n_cupl = cupon_l.shape[1] - 1
    n_descl = descuento_l.shape[1] - 1
    n_cupc = cupon_c.shape[1] - 1
    n_descc = descuento_c.shape[1] - 1

    # 
    X = cupon_l.merge(descuento_l,how = 'inner',on = 'DATE')
    X = X.merge(cupon_c,how = 'inner',on = 'DATE')
    X = X.merge(descuento_c,how = 'inner',on = 'DATE')
    X.set_index('DATE', inplace=True)

    Y_s = pd.DataFrame()
    irs_c= InterestRateSwap()
    cup_l = X.iloc[:,:n_cupl]
    desc_l = X.iloc[:,n_cupl:n_cupl+n_descl]
    cup_c = X.iloc[:,n_cupl+n_descl:n_cupl+n_descl+n_cupc]
    desc_c = X.iloc[:,n_cupl+n_descl+n_cupc:n_cupl+n_descl+n_cupc+n_descc]

    # Largo
    largo = []
    for i in range(len(cup_l)):
      fval_aux = cup_l.index[i]
      flotante =  cup_l.iloc[i,:].values/100
      flotante = list(flotante)
      descuentos = desc_l.iloc[i,:].values/100
      descuentos = list(descuentos)
      calen_flotante_l_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_l]
      calen_desc_l_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_l]
      irs = InterestRateSwap()
      # fval vs fval_aux
      irs.compute(fval_aux, posiciones[0], n_l,tasas_fijas[0] , flotante,calen_flotante_l_aux, descuentos, calen_desc_l_aux, plazos_l, tau_l)
      largo.append(irs.precio)

    # Corto
    corto = []
    for i in range(len(cup_c)):
      fval_aux = cup_c.index[i]
      flotante =  cup_c.iloc[i,:].values/100
      flotante = list(flotante)
      descuentos = desc_c.iloc[i,:].values/100
      descuentos = list(descuentos)
      calen_flotante_c_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_c]
      calen_desc_c_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_c]
      irs = InterestRateSwap()
      # fval vs fval_aux
      irs.compute(fval_aux, posiciones[1], n_c, tasas_fijas[1] , flotante,calen_flotante_c_aux, descuentos, calen_desc_c_aux, plazos_c, tau_c)
      corto.append(irs.precio)

    Y_s['IRS_l'] = largo
    Y_s['IRS_c'] = corto

    X0 = X.head(1)
    DeltaX = np.log(X.shift(-1, fill_value=1)/X)
    X_s=np.exp(DeltaX.squeeze())*X0.squeeze()

    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    print(Y0)
    #PL = Y0 - Y_s
    PL = Y_s - Y0
    cols=PL.shape[1]
    PL['Total']=PL.sum(axis=1)
    VaR=PL.quantile(alpha, numeric_only=False)
    VaR_C=[]

    for i in range(0,PL.shape[1]):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)
    
    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = {'Largo':posiciones[0],'Corto':posiciones[1]}
    self.tasas_fijas = {'Largo':tasas_fijas[0],'Corto':tasas_fijas[1]}
    self.vencimientos = {'Largo':vencimientos[0],'Corto':vencimientos[1]}
    self.dias_bono = {'Largo':dias_bono[0],'Corto':dias_bono[1]}
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    return


In [ ]:
# Actualizar datos con los de la tarea una vez nos den las nuevas tasas
fval = datetime(2020,3,6) # l,c

tasas_fijas = [0.075,0.079]
cupon_l = pd.read_csv("/tasa_DIRS_SW_OP.txt", delimiter = "\t")
cupon_l = cupon_l.rename(columns = {'DATE ':'DATE'})
cupon_l['DATE'] =cupon_l['DATE'].apply(lambda x: str(x))
cupon_l['DATE'] = cupon_l['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
cupon_c  = cupon_l

descuento_l = pd.read_csv("/tasa_TIIE_SW_OP.txt", delimiter = "\t")
descuento_l = descuento_l.rename(columns = {'DATE ':'DATE'})
descuento_l['DATE'] =descuento_l['DATE'].apply(lambda x: str(x))
descuento_l['DATE'] = descuento_l['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
descuento_c = descuento_l

vencimientos = [360,588]
dias_bono = [28,28]
posiciones = [100000, -1600]
alpha = 0.98

In [ ]:
miIRS= Riesgo_IRS()

In [ ]:
miIRS.compute(fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c, vencimientos, dias_bono, posiciones, alpha)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'30_x', '270_x', '2160_x', '180_x', '90_x', '1_x', '2880_x', '3600_x', '7_x', '360_x', '2520_x', '720_x', '1440_x', '1800_x', '3240_x', '1080_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[438.27403382   5.55170275]


In [ ]:
miIRS.summary

,VaR,CVaR
IRS_l,123.684954,157.933410
IRS_c,10.265112,10.640347
Total,118.546607,151.917786


In [ ]:
print(miIRS.posiciones)
print(miIRS.tasas_fijas)
print(miIRS.vencimientos)
print(miIRS.dias_bono)

{'Largo': 100000, 'Corto': -1600}
{'Largo': 0.075, 'Corto': 0.079}
{'Largo': 360, 'Corto': 588}
{'Largo': 28, 'Corto': 28}


# Caplet y Floorlet

1.6
*    Tasa de interés larga call con strike de 5.8 %, vencimiento de 1700 días y 1000 contratos.
*    Put larga de tasa de interés con strike de 6.0 % de la tiie nocional de 500 contratos y vencimiento 700 días.



In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

In [ ]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
# require(data.table)
install.packages("data.table")
library(data.table)

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}


diagm=function(x)		#función para diagonalizar una matriz en una de mayor dimensión
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  if(m01>=n01 )
  {
    res=matrix(0,m01,m01)
    for (i in 1:(m01/n01))
    {
      z=(i-1)*n01
      for(j in 1:n01)
      {
        for(k in 1:n01)
        {
          res[z+k,z+j]=x[k,z+j]
        }
      }
    }
    res
  }
  else
  {
    "no es matriz cuadrada"
  }
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [ ]:
%%R

#MEDIDAS DE RIESGO CON ALISADO

#VaR con alisado
wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


 Definición de parámetros para valorar:

In [ ]:
%%R
fval=as.Date("20220331",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia
#yext=1 #si se usa la historia de internet o fija

btasadesc_oir="RiesgosFinancieros/2022-2/Tarea/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2022-2/Tarea/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2022-2/Tarea/tvoltiie_opc.txt"
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.058, 0.06)
contratos_oir=cbind(1000, 500)
nominal_oir=1 
cs_oir=1 #1 si es continua la tasa 0 si es simple

Carga de datos:

In [ ]:
%%R

#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])


 Integración de Insumos:
 
 Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [ ]:
%%R
#INTERSECCIÓN DE FECHAS DE LAS OPCIONES    
  
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(x1_orig_oir[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


In [ ]:
%%R
head(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp)
head(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
#Hace match con las fechas de las tasas (factores de riesgo)

In [ ]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)



Valuación de las opciones:

In [ ]:
%%R


##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

#Se toma la primera fila de las tasas, ie la tasa más actual para calcular la valoracion
x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#Usa la funcion para valuar las opciones
V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
V0_oir


         [,1]       [,2]
[1,] 1.416363 0.06098592


Integración de factores y cálculo de varianza-covarianza:

In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if=matrix(0,1,1)
n_if[1]=ncol(X_oir) #fwd tdc

#valor del portafolio

V0_port=cbind(V0_oir)
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(X_oir) #Factores de riesgo del portafolios de 2 opciones y 3 factores de riesgo

#Tasas que nos interesan 
#print(head(X_port))


#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1),]/X_port[2:(n),]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
DeltaX_s=DeltaX_port

###Riesgo a nivel contrato SIN ALISADO (Opciones)

In [ ]:
%%R
m=ncol(plazos_oir)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s  #PASO CLAVE
x0_oir=X_oir[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}



#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}


#VaR Total
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


print("Valor del portafolio")
print(V0_oir)
print("VaR por contrato")
print(VaRCont_oir)
print("VaR por el factor de riesgo tasas de descuento por contrato")
print(VaRfr1_oir)
print("VaR por el factor de riesgo tasas spot por contrato")
print(VaRfr2_oir)
print("VaR por el factor de riesgo volatilidades por contrato")
print(VaRfr3_oir)
print("VaR por el factor de riesgo tasas de descuento total")
print(VaRTotalfr1_oir)
print("VaR por el factor de riesgo tasas spot 2 total")
print(VaRTotalfr2_oir)
print("VaR por el factor de riesgo volatilidades total")
print(VaRTotalfr3_oir)
print("VaR Total")
print(VaRTotal_oir)

print("CVaR por contrato")
print(CVaRCont_oir)
print("CVaR por el factor de riesgo tasas de descuento por contrato")
print(CVaRfr1_oir)
print("CVaR por el factor de riesgo tasas spot por contrato")
print(CVaRfr2_oir)
print("CVaR por el factor de riesgo volatilidades por contrato")
print(CVaRfr3_oir)
print("CVaR por el factor de riesgo tasas de descuento total")
print(CVaRTotalfr1_oir)
print("CVaR por el factor de riesgo tasas spot total")
print(CVaRTotalfr2_oir)
print("CVaR por el factor de riesgo volatilidades total")
print(CVaRTotalfr3_oir)
print("CVaR Total")
print(CVaRTotal_oir)


         [,1]       [,2]
[1,] 1.416363 0.06098592
[1] "VaR por contrato"
            [,1]        [,2]
[1,] -0.07125505 -0.01027072
[1] "VaR por factor de riesgo 1 por contrato"
             [,1]          [,2]
[1,] -0.007026946 -8.401139e-05
[1] "VaR por factor de riesgo 2 por contrato"
            [,1]       [,2]
[1,] -0.06951811 -0.0102704
[1] "VaR por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]    0    0
[1] "VaR por factor de riesgo 1 total"
[1] -0.007089484
[1] "VaR por factor de riesgo 2 total"
[1] -0.06483398
[1] "VaR por factor de riesgo 3 total"
[1] 0
[1] "VaR Total"
[1] -0.06483406
[1] "CVaR por contrato"
            [,1]        [,2]
[1,] -0.09015167 -0.01481039
[1] "CVaR por factor de riesgo 1 por contrato"
             [,1]         [,2]
[1,] -0.007435223 -0.000160021
[1] "CVaR por factor de riesgo 2 por contrato"
            [,1]        [,2]
[1,] -0.09013672 -0.01481013
[1] "CVaR por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]  NaN  NaN
[1] "CVaR por factor

###Riesgo a nivel contrato CON ALISADO (Opciones)

Generación de escenarios:

In [ ]:
%%R
w0 <- 0.05
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root


#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

p_esc=genera_esc(lambda,w0,Ns)

In [ ]:
%%R
m=ncol(plazos_oir)
VaRCACont_oir=matrix(0,1,m)
VaRCAfr1_oir=matrix(0,1,m)
VaRCAfr2_oir=matrix(0,1,m)
VaRCAfr3_oir=matrix(0,1,m)
CVaRCACont_oir=matrix(0,1,m)
CVaRCAfr1_oir=matrix(0,1,m)
CVaRCAfr2_oir=matrix(0,1,m)
CVaRCAfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_oir[i]=wquantile(PG_oir[,i],p_esc,1-alpha)
  VaRCAfr1_oir[i]=wquantile(PGfr1_oir[,i],p_esc,1-alpha)
  VaRCAfr2_oir[i]=wquantile(PGfr2_oir[,i],p_esc,1-alpha)
  VaRCAfr3_oir[i]=wquantile(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCAfr1_oir[i]= wcvar(PGfr1_oir[,i],p_esc,1-alpha)
  CVaRCAfr2_oir[i]= wcvar(PGfr2_oir[,i],p_esc,1-alpha)
  CVaRCAfr3_oir[i]= wcvar(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCACont_oir[i]= wcvar(PG_oir[,i],p_esc,1-alpha)
}


#VaRCA Total
VaRCATotal_oir=wquantile(PGT_oir,p_esc,1-alpha)
CVaRCATotal_oir= wcvar(PGT_oir,p_esc,1-alpha)
VaRCATotalfr1_oir=wquantile(PGfr1T_oir,p_esc,1-alpha)
CVaRCATotalfr1_oir= wcvar(PGfr1T_oir,p_esc,1-alpha)
VaRCATotalfr2_oir=wquantile(PGfr2T_oir,p_esc,1-alpha)
CVaRCATotalfr2_oir= wcvar(PGfr2T_oir,p_esc,1-alpha)
VaRCATotalfr3_oir=wquantile(PGfr3T_oir,p_esc,1-alpha)
CVaRCATotalfr3_oir= wcvar(PGfr3T_oir,p_esc,1-alpha)

print("Valor del portafolio")
print(V0_oir)
print("VaRCA por contrato")
print(VaRCACont_oir)
print("VaRCA por el factor de riesgo tasas de descuento por contrato")
print(VaRCAfr1_oir)
print("VaRCA por el factor de riesgo tasas spot por contrato")
print(VaRCAfr2_oir)
print("VaRCA por el factor de riesgo volatilidades por contrato")
print(VaRCAfr3_oir)
print("VaRCA por el factor de riesgo tasas de descuento total")
print(VaRCATotalfr1_oir)
print("VaRCA por el factor de riesgo tasas spot total")
print(VaRCATotalfr2_oir)
print("VaRCA por el factor de riesgo volatilidades total")
print(VaRCATotalfr3_oir)
print("VaRCA Total")
print(VaRCATotal_oir)

print("CVaRCA por contrato")
print(CVaRCACont_oir)
print("CVaRCA por el factor de riesgo tasas de descuento por contrato")
print(CVaRCAfr1_oir)
print("CVaRCA por el factor de riesgo tasas spot por contrato")
print(CVaRCAfr2_oir)
print("CVaRCA por el factor de riesgo volatilidades por contrato")
print(CVaRCAfr3_oir)
print("CVaRCA por el factor de riesgo tasas de descuento total")
print(CVaRCATotalfr1_oir)
print("CVaRCA por el factor de riesgo tasas spot total")
print(CVaRCATotalfr2_oir)
print("CVaRCA por el factor de riesgo volatilidades total")
print(CVaRCATotalfr3_oir)
print("CVaRCA Total")
print(CVaRCATotal_oir)

         [,1]       [,2]
[1,] 1.416363 0.06098592
[1] "VaRCA por contrato"
            [,1]         [,2]
[1,] -0.06023641 -0.007223141
[1] "VaRCA por factor de riesgo 1 por contrato"
            [,1]          [,2]
[1,] -0.00414018 -4.939062e-05
[1] "VaRCA por factor de riesgo 2 por contrato"
            [,1]        [,2]
[1,] -0.05640292 -0.00719673
[1] "VaRCA por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]    0    0
[1] "VaRCA por factor de riesgo 1 total"
[1] -0.004164824
[1] "VaRCA por factor de riesgo 2 total"
[1] -0.05169848
[1] "VaRCA por factor de riesgo 3 total"
[1] 0
[1] "VaRCA Total"
[1] -0.05555714
[1] "CVaRCA por contrato"
            [,1]        [,2]
[1,] -0.09902542 -0.01145685
[1] "CVaRCA por factor de riesgo 1 por contrato"
             [,1]          [,2]
[1,] -0.005424767 -6.094306e-05
[1] "CVaRCA por factor de riesgo 2 por contrato"
            [,1]        [,2]
[1,] -0.09699062 -0.01144952
[1] "CVaRCA por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]  N

# Cross Currency Swap

2. Crear un programa en google colab (R o Python) para valorar portafolio de CCS peso mexicano - dólar tasa variable vs variable, y que calcule el riesgo por simulación histórica (2.5 puntos extra).

In [ ]:
class CrossCurrencySwap:
    
    def __init__(self):
        self.precio = None
        self.nocional_extranjera = None
        self.nocional_local = None
        self.n_cupones = None
        self.posicion = None
        self.convencion = None
        self.flotante_ext = None
        self.flotante_loc = None
        self.descuentos_ext = None
        self.descuentos_loc = None
        self.spot = None
        self.summary = None
        
    def compute(self, fval, n,M_l, M_e, Spot, flot_l,cal_flot_l, flot_e,cal_flot_e, desc_l, cal_desc_l, 
                desc_e, cal_desc_e,plazos, tau,  payer = False, act_360 = True):
        """
        M: Nocional
        n: # de cupones
        fix_rate: Tasa fija de la pata larga del IRS
        flotante: Curva de tasas de mercado (la que encuentras en BANXICO) de la pata corta
        calen_flotante: Las fechas dadas por la curva de flotante
        descuentos: Curca de tasas de descuento de mercado para valuar el IRS
        calen_desc: Las fechas dadas por la curva de descuentos
        plazos : Lista de plazo del i -ésimo cupón (habiles)
        tau: Plazo en dias del i-ésimo cupón. (naturales)
        payer: Dummy; 1 si paga fija, 0 si paga flotante
        act_360: Dummy; tipo de convención 1 si act/360, 0 si act/365
        """

        spot = fval
        
        if act_360:
            conv = 360
        else:
            conv = 365
            
        aux = pd.DataFrame({"Cupon":list(range(1,n+1)), "Tau":tau})
        aux["Desde Spot"] = aux["Tau"].cumsum()
        aux["Fecha"] = aux["Desde Spot"].apply(lambda x: timedelta(x) + spot)
                
        df = pd.DataFrame({"Cupon":list(range(1,n+1)), "Plazo": plazos, "Tau": tau})
                
        df["Fechas"] =  aux["Fecha"]
        df["Flotante_Local"] = df["Fechas"].apply(InterpolacionLineal, args=(cal_flot_l, flot_l))
        df["Descuentos_Local"]=df["Fechas"].apply(InterpolacionLineal, args=(cal_desc_l, desc_l))
        df["Flotante_Ext"] = df["Fechas"].apply(InterpolacionLineal, args=(cal_flot_e, flot_e))
        df["Descuentos_Ext"]=df["Fechas"].apply(InterpolacionLineal, args=(cal_desc_e, desc_e))
        
        # Calculo de tasas forward
        df['Forward_Flotante_Local'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante_Local']*aux['Desde Spot']/conv))/(1+(df['Flotante_Local'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante_Local'][0] = df['Flotante_Local'][0]
        df['Forward_Flotante_Ext'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante_Ext']*aux['Desde Spot']/conv))/(1+(df['Flotante_Ext'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante_Ext'][0] = df['Flotante_Ext'][0]
        
        df["Sumando_Local"] = (M_l*((df["Forward_Flotante_Local"])*df["Plazo"]/conv)) / (1 + (df["Descuentos_Local"]*df["Tau"]/conv))
        df["Sumando_Ext"] = (M_e*Spot*((df["Forward_Flotante_Ext"])*df["Plazo"]/conv)) / (1 + (df["Descuentos_Ext"]*df["Tau"]/conv))
        df["Sumando"] = df["Sumando_Local"] - df["Sumando_Ext"]

        self.precio =  ((-1)**payer)*df["Sumando"].sum()
        self.nocional_extranjera = M_e
        self.nocional_local = M_l
        self.n_cupones = n
        self.posicion = "Payer" if payer else "Receiver"
        self.convencion = conv
        self.flotante_ext = df[["Fechas","Forward_Flotante_Ext"]]
        self.flotante_loc = df[["Fechas","Forward_Flotante_Local"]]
        self.descuentos_ext = df[["Fechas", "Descuentos_Ext"]]
        self.descuentos_loc = df[["Fechas", "Descuentos_Local"]]
        self.spot = Spot
        self.summary = df
        return
    
    # PENDIENTE
    def plots(self):
        Bool = True
        while Bool:
            print("\n")
            choice = input("Si deseas ver las tasas de descuento teclea 'Descuentos', si deseas\
                                ver las tasas flotantes teclea 'Tasas' o si deseas salir escribe '0':\n")
            if choice == 'Descuentos' or choice =='Tasas' or choice == '0':
                Bool = False
            else:
                print('\nTu elección no es válida')
        if choice == 'Descuentos':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Descuentos'])
            ax.set_title("Curva de Descuentos",fontsize = '25')
            plt.show()
            
            return
        elif choice == 'Tasas':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Forward_Flotante'], color = 'red')
            ax.set_title("Curva de Tasas Forward Flotantes",fontsize = '25')
            plt.show()
            
            return
        else:
            return

In [ ]:
class Riesgo_CCS:

  def __init__(self):
    self.nocionales = None
    self.vencimiento = None
    self.dias_bono = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.spot = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    
  def compute(self, fval, spot,cupon_l, descuento_l,cupon_e,descuento_e, vencimiento, dias_bono, posiciones, alpha):
    """
    a
    """
    calen_flotante_l = cupon_l.columns
    calen_flotante_l = list(calen_flotante_l)
    calen_flotante_l.remove('DATE')
    calen_desc_l = descuento_l.columns
    calen_desc_l = list(calen_desc_l)
    calen_desc_l.remove('DATE')


    calen_flotante_e = cupon_e.columns
    calen_flotante_e = list(calen_flotante_e)
    calen_flotante_e.remove('DATE')
    calen_desc_e = descuento_e.columns
    calen_desc_e = list(calen_desc_e)
    calen_desc_e.remove('DATE')


    n = int(vencimiento/dias_bono)
    if n == (vencimiento/dias_bono):
        plazos = [dias_bono]*n
        tau = plazos
    else:
        plazos = [vencimiento-n*dias_bono] + [dias_bono]*(n-1)
        tau = plazos

    # Aquí vamos
    n_cupl = cupon_l.shape[1] - 1
    n_descl = descuento_l.shape[1] - 1
    n_cupe = cupon_e.shape[1] - 1
    n_desce = descuento_e.shape[1] - 1

    # Spot
    cross_currency = yf.download(spot, end = fval)
    cross_currency = cross_currency[['Close']]
    cross_currency = cross_currency.iloc[::-1]
    cross_currency =cross_currency.reset_index()
    cross_currency = cross_currency.rename(columns = {'Date':'DATE','Close':spot})
    
    #  X

    X = cupon_l.merge(descuento_l,how = 'inner',on = 'DATE')
    X = X.merge(cupon_e,how = 'inner',on = 'DATE')
    X = X.merge(descuento_e,how = 'inner',on = 'DATE')
    X = X.merge(cross_currency, how = 'inner',on = 'DATE')
    X.set_index('DATE', inplace=True)

    Y_s = pd.DataFrame()
    irs_c= InterestRateSwap()
    cup_l = X.iloc[:,:n_cupl]
    desc_l = X.iloc[:,n_cupl:n_cupl+n_descl]
    cup_e = X.iloc[:,n_cupl+n_descl:n_cupl+n_descl+n_cupe]
    desc_e = X.iloc[:,n_cupl+n_descl+n_cupe:n_cupl+n_descl+n_cupe+n_desce]

    #(self, fval, n,M_l, M_e, Spot, flot_l,cal_flot_l, flot_e,cal_flot_e, descu_l, cal_desc_l, 
    #            desc_e, cal_desc_e,plazos, tau,  payer = False, act_360 = True)

    # Valoraciones
    aux = []
    for i in range(len(cup_l)):
      fval_aux = cup_l.index[i]
      flot_l =  cup_l.iloc[i,:].values/100
      flot_l = list(flot_l)
      descu_l = desc_l.iloc[i,:].values/100
      descu_l = list(descu_l)
      cal_flot_l_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_l]
      cal_desc_l_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_l]

      flot_e =  cup_e.iloc[i,:].values/100
      flot_e = list(flot_e)
      descu_e = desc_e.iloc[i,:].values/100
      descu_e = list(descu_e)
      cal_flot_e_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_e]
      cal_desc_e_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_e]

      ccs = CrossCurrencySwap()
      ccs.compute(fval_aux, n ,posiciones[0],posiciones[1], X[spot][i], flot_l,cal_flot_l_aux, flot_e,cal_flot_e_aux, descu_l, cal_desc_l_aux,\
                  descu_e, cal_desc_e_aux,plazos, tau)

      aux.append(ccs.precio)

    Y_s['CCS'] = aux

    X0 = X.head(1)
    DeltaX = np.log(X.shift(-1, fill_value=1)/X)
    X_s=np.exp(DeltaX.squeeze())*X0.squeeze()

    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    print(Y0)
    #PL = Y0 - Y_s
    PL = Y_s - Y0
    cols=PL.shape[1]
    PL['Total']=PL.sum(axis=1)
    VaR=PL.quantile(alpha, numeric_only=False)
    VaR_C=[]

    for i in range(0,PL.shape[1]):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)
    
    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.nocionales = {'Local':posiciones[0],'Extranjero':posiciones[1]}
    self.vencimiento = vencimiento
    self.dias_bono = dias_bono
    self.spot = spot
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    return
